In [ ]:
import requests
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
from io import BytesIO

TMDB_API_KEY = "1dc3598610c5a9627425e69414f22ce2"

# Data
data = [
    {"rank": 1, "awardyear": 2019, "winner_title": "Sally4Ever", "winner_tconst": "tt8358050", "top_nominee_title": "Derry Girls", "top_nominee_tconst": "tt7120662", "rating_diff": -1.9},
    {"rank": 2, "awardyear": 2024, "winner_title": "Such Brave Girls", "winner_tconst": "tt18974452", "top_nominee_title": "Big Boys", "top_nominee_tconst": "tt13683866", "rating_diff": -1.1},
    {"rank": 3, "awardyear": 2012, "winner_title": "Mrs. Brown's Boys", "winner_tconst": "tt1819022", "top_nominee_title": "Friday Night Dinner", "top_nominee_tconst": "tt1844923", "rating_diff": -1.1},
    {"rank": 4, "awardyear": 2020, "winner_title": "Stath Lets Flats", "winner_tconst": "tt7524382", "top_nominee_title": "Fleabag", "top_nominee_tconst": "tt5687612", "rating_diff": -1.0},
    {"rank": 5, "awardyear": 2013, "winner_title": "Twenty Twelve", "winner_tconst": "tt1861225", "top_nominee_title": "The Thick of It", "top_nominee_tconst": "tt0459159", "rating_diff": -1.0}
]

# Poster helpers
def get_tmdb_poster(title):
    url = f"https://api.themoviedb.org/3/search/tv?api_key={TMDB_API_KEY}&query={title}"
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        data = r.json()
        if data["results"]:
            poster_path = data["results"][0].get("poster_path")
            if poster_path:
                return f"https://image.tmdb.org/t/p/w500{poster_path}"
    except:
        return None
    return None

def download_image(url):
    if url:
        try:
            r = requests.get(url, timeout=10)
            return Image.open(BytesIO(r.content)).convert("RGBA")
        except:
            pass
    return Image.new("RGBA", (500, 750), color=(220, 220, 220, 255))

def apply_fade(image, strength=0.7):
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(strength)

# Download posters
for row in data:
    row["winner_poster"] = download_image(get_tmdb_poster(row["winner_title"]))
    row["nominee_poster"] = apply_fade(download_image(get_tmdb_poster(row["top_nominee_title"])))

# Layout
poster_height = 0.17
poster_width = 0.12
y_gap = 0.025
row_height = poster_height + y_gap
total_rows = len(data)

fig_height = total_rows * row_height + 0.02  # small padding
fig, ax = plt.subplots(figsize=(14, fig_height * 10))  # 10x scale

ax.set_xlim(0, 1)
ax.set_ylim(0, fig_height)
ax.axis("off")

# Draw rows
for i, row in enumerate(data):
    y_top = fig_height - i * row_height
    y_bot = y_top - poster_height
    y_center = (y_top + y_bot) / 2

    # Horizontal line
    ax.plot([0, 1], [y_top + y_gap / 2, y_top + y_gap / 2], color="black", linewidth=1.5)

    # Rank
    ax.text(0.04, y_center, str(row["rank"]), fontsize=60, fontweight="bold", fontname="DejaVu Serif", ha="center", va="center")

    # Winner poster
    ax.imshow(row["winner_poster"], extent=(0.07, 0.07 + poster_width, y_bot, y_top))

    # Nominee poster
    ax.imshow(row["nominee_poster"], extent=(0.26, 0.26 + poster_width, y_bot, y_top))

    # Difference
    ax.text(0.47, y_center, f"{row['rating_diff']:+.1f}", fontsize=26, ha="left", va="center", weight="bold", color="red")

    # Year
    ax.text(0.94, y_center, str(row["awardyear"]), fontsize=52, ha="center", va="center", weight="bold")

# Final layout
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
plt.savefig("bafta_top5_ranked_cleaned.png", dpi=300)
plt.show()
